In [15]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim

if torch.cuda.is_available():
    torch.backends.cudnn.deterministic = True
    device = torch.device("cuda:0")
    
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
print(torch.cuda.is_available())
print("Device:", device)

True
Device: cuda:0


In [16]:
#Config
RANDOM_SEED = 1
batch_size = 128
epochs = 30
number_classes = 10
learn_rate = 0.001

In [17]:
torch.manual_seed(RANDOM_SEED)
#Batchsize auf 128 anpassen.


trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

#classes = ('plane', 'car', 'bird', 'cat',
#           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [18]:
class AlexNet(nn.Module):
    def __init__(self, num_classes):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=64,
                      kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            nn.Conv2d(in_channels=64, out_channels=192,
                      kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            nn.Conv2d(in_channels=192, out_channels=384,
                      kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            
            nn.Conv2d(in_channels=384, out_channels=256,
                      kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            
            nn.Conv2d(in_channels=256, out_channels=256,
                      kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), 256 * 6 * 6)
        x = self.classifier(x)
        return x

net = AlexNet(number_classes).to(device) 


In [19]:
import keyboard
import time

def simulate_key_press(key):
    keyboard.press(key)
    time.sleep(0.1) 
    keyboard.release(key)

In [20]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=learn_rate, momentum=0.9)

In [21]:
# Funktion zur Berechnung der Genauigkeit auf den Testdaten
def calculate_accuracy(loader):
    correct = 0
    total = 0
    with torch.no_grad():
        for data in loader:
            inputs, labels = data[0].to(device), data[1].to(device)
            outputs = net(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return correct / total

time.sleep(1)
simulate_key_press('p')
start_time = time.time()

for epoch in range(epochs):  
    epoch_start_time = time.time()
    for i, data in enumerate(trainloader, 0):
        # Verschiebe die daten auf die GPU
        inputs, labels = data[0].to(device), data[1].to(device)  

        # Setze parameter gradients auf 0
        optimizer.zero_grad()
    
        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print Statistiken
        if i % 2000 == 1999:    
            print('[%d, %5d]' %
                  (epoch + 1, i + 1))

    
    epoch_end_time = time.time()
    elapsed_time = epoch_end_time - epoch_start_time
    print(f"Epoch {epoch + 1}/{epochs}, Elapsed Time: {elapsed_time:.2f} seconds")

total_training_time = time.time() - start_time

# Berechnung der Genauigkeit auf den Testdaten
total_accuracy = calculate_accuracy(testloader) * 100
print(f"Accuracy on test data: {total_accuracy:.2f}")
print(f"Total Training Time: {total_training_time:.2f} seconds")
print('Finished Training')

time.sleep(1)
simulate_key_press('p')

Epoch 1/30, Elapsed Time: 16.85 seconds
Epoch 2/30, Elapsed Time: 16.78 seconds
Epoch 3/30, Elapsed Time: 16.75 seconds
Epoch 4/30, Elapsed Time: 16.62 seconds
Epoch 5/30, Elapsed Time: 16.58 seconds
Epoch 6/30, Elapsed Time: 16.60 seconds
Epoch 7/30, Elapsed Time: 16.60 seconds
Epoch 8/30, Elapsed Time: 16.77 seconds
Epoch 9/30, Elapsed Time: 16.62 seconds
Epoch 10/30, Elapsed Time: 16.59 seconds
Epoch 11/30, Elapsed Time: 16.78 seconds
Epoch 12/30, Elapsed Time: 16.62 seconds
Epoch 13/30, Elapsed Time: 16.57 seconds
Epoch 14/30, Elapsed Time: 16.61 seconds
Epoch 15/30, Elapsed Time: 16.78 seconds
Epoch 16/30, Elapsed Time: 16.60 seconds
Epoch 17/30, Elapsed Time: 16.60 seconds
Epoch 18/30, Elapsed Time: 16.60 seconds
Epoch 19/30, Elapsed Time: 16.58 seconds
Epoch 20/30, Elapsed Time: 16.61 seconds
Epoch 21/30, Elapsed Time: 16.59 seconds
Epoch 22/30, Elapsed Time: 16.61 seconds
Epoch 23/30, Elapsed Time: 16.78 seconds
Epoch 24/30, Elapsed Time: 16.61 seconds
Epoch 25/30, Elapsed Time

In [40]:
import csv
import os

#Zum loggen der Trainingslaufzeit und accuracy  des Models
def add_row_to_csv(filename, datensatz_name, frequency, 
                   voltage, runtime, accuracy):
    # Daten für die neue Zeile
    new_row = {'Datensatz': datensatz_name, 'Frequenz': frequency, 
               'voltage': voltage, 'Laufzeit': runtime, 
               'Accuracy': accuracy}
    
    # Überprüfe, ob die Datei existiert
    file_exists = os.path.isfile(filename)
    
    # Öffne die CSV-Datei im Modus 'a' (append), um Werte hinzuzufügen
    with (open(filename, 'a', newline='') as csvfile):
        # Erstelle einen CSV-Writer, falls die Datei neu erstellt wird
        writer = csv.DictWriter(
            csvfile, fieldnames=['Datensatz', 'Frequenz', 
                               'voltage', 'Laufzeit', 'Accuracy'])
        
        # Schreibe die Kopfzeile, falls die Datei neu erstellt wurde
        if not file_exists:
            writer.writeheader()
        
        # Schreibe die neue Zeile in die CSV-Datei
        writer.writerow(new_row)

filename = 'alexnet_cifar_spannung.csv'
datensatz_name = 'Cifar10'
frequency = '3600'
voltage = '1.1'
runtime = round(total_training_time, 2)
accuracy = round(total_accuracy, 2)

add_row_to_csv(filename, datensatz_name, frequency, 
               voltage, runtime, accuracy)
